In [1]:
import numpy as np
from sparse_vector import *
from vector_polynomial_isomorphism import *

We note that we are using the base field `QQ` for all computations currently

In [2]:
# Let us initialize constants and vector spaces.

N = 16
d = 3
indeterminates = [var("".join(["x_", str(i), str(j), str(k)])) for i in range(2) for j in range(2) for k in range(4)]
R = PolynomialRing(QQ, indeterminates)
R.inject_variables()

glN = MatrixSpace(QQ, N)
dimV = binomial(N + d - 1, d)
V = VectorSpace(QQ, dimV)

vp_iso = vector_polynomial_isomorphism(R, V, indeterminates, d)

Defining x_000, x_001, x_002, x_003, x_010, x_011, x_012, x_013, x_100, x_101, x_102, x_103, x_110, x_111, x_112, x_113


In [3]:
# construct f

M_4by4 = MatrixSpace(R, 4, 4)

x_sub = lambda i,j,k: R(var("".join(["x_", str(i), str(j), str(k)])))

A_12 = M_4by4(0)

for i in range(2):
    for j in range(2):
        for k in range(3):
            A_12[2*i + j, k] = x_sub(i,j,k)

f = A_12.minors(3)

f_vec = [vp_iso.polynomial_to_vector(fi) for fi in f]

In [4]:
# define W = span{f}, W_perp = orthogonal complement of span{f}

W = V.subspace(f_vec)

f_vec_basis = W.basis() # take only the linearly independent ones
    
grad_f = [vp_iso.vector_to_polynomial(fi).gradient() for fi in f_vec_basis]

W_perp = W.complement()

print(W)
print(W_perp)

g = W_perp.basis()
p = W.dimension()
q = W_perp.dimension()

Vector space of degree 816 and dimension 4 over Rational Field
Basis matrix:
4 x 816 dense matrix over Rational Field
Vector space of degree 816 and dimension 812 over Rational Field
Basis matrix:
812 x 816 dense matrix over Rational Field


In [5]:
# action of basis matrix E_ab in glN onto homogeneous polynomial f[i]
def E_action(a, b, i):
    sum = -1*indeterminates[b]*grad_f[i][a]
    return R(sum)

We note `M_f` $:= \tilde{M}_f$

In [6]:
# Pre-processing memoization of entries of M_f

# E_action_f[l][i][j] = (E_ij)(f_l)
E_action_f = [[[0 for j in range(N)] for i in range(N)] for l in range(p)]
for l in range(p):
    for i in range(N):
        for j in range(N):
            E_action_f[l][i][j] = sparsify(vp_iso.polynomial_to_vector(E_action(i,j,l)))

# convert g to sparse representation
g_sparse = [sparsify(g[i]) for i in range(q)]

In [8]:
# Construct M_f

M_f = zero_matrix(QQ, p*q, N^2)
for l in range(p):
    for k in range(q):
        for i in range(N):
            for j in range(N):
                M_f[(l*p) + k,(N*i) + j] = sparse_inner_product(E_action_f[l][i][j], g_sparse[k])

In [9]:
kern = M_f.right_kernel()
print([sparsify(x) for x in kern.basis()])
print(len(kern.basis()))
print('rank(M_f) =', M_f.rank())
print('dim(ker(M_f)) =', glN.dimension() - M_f.rank())

[[(0, 1)], [(1, 1)], [(2, 1)], [(3, 1)], [(4, 1)], [(5, 1)], [(6, 1)], [(7, 1)], [(8, 1)], [(9, 1)], [(10, 1)], [(11, 1)], [(12, 1)], [(13, 1)], [(14, 1)], [(15, 1)], [(16, 1)], [(17, 1)], [(18, 1)], [(19, 1)], [(20, 1)], [(21, 1)], [(22, 1)], [(23, 1)], [(24, 1)], [(25, 1)], [(26, 1)], [(27, 1)], [(28, 1)], [(29, 1)], [(30, 1)], [(31, 1)], [(32, 1)], [(33, 1)], [(34, 1)], [(35, 1)], [(36, 1)], [(37, 1)], [(38, 1)], [(39, 1)], [(40, 1)], [(41, 1)], [(42, 1)], [(43, 1)], [(44, 1)], [(45, 1)], [(46, 1)], [(47, 1)], [(48, 1)], [(49, 1)], [(50, 1)], [(51, 1)], [(52, 1)], [(53, 1)], [(54, 1)], [(55, 1)], [(56, 1)], [(57, 1)], [(58, 1)], [(59, 1)], [(60, 1)], [(61, 1)], [(62, 1)], [(63, 1)], [(64, 1), (81, 1), (98, 1)], [(68, 1), (153, -1), (170, -1), (221, -1), (238, -1)], [(69, 1), (137, 1), (205, 1)], [(70, 1), (138, 1), (206, 1)], [(72, 1), (89, 1), (106, 1)], [(76, 1), (93, 1), (110, 1)], [(84, 1), (152, 1), (220, 1)], [(85, 1), (153, 1), (221, 1)], [(86, 1), (154, 1), (222, 1)], [(100,

In [10]:
print((2^2-1) + (2^2-1) + (4^2-1) +1)

22
